In [6]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from langgraph.graph import StateGraph,START,END

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [4]:
class LLMState(TypedDict):
    question:str
    answer:str

In [5]:
def llm_qa(state:LLMState) -> LLMState:

    #extract question
    ques = state['question']

    #form a prompt
    prompt = f'Answer the following question{ques}'

    #ask to llm
    ans = model.invoke(prompt).content

    #update the state
    state['answer'] = ans
    return state



In [8]:
#create our graph
graph = StateGraph(LLMState)

#add nodes
graph.add_node('llm_qa',llm_qa)

#add edge
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

workflow = graph.compile()


In [9]:
#execute
initial_state = {'question':"How far is moon from the earth?"}
final_state = workflow.invoke(initial_state)
print(final_state)

{'question': 'How far is moon from the earth?', 'answer': "The distance between the Earth and the Moon isn't constant, as the Moon's orbit is an ellipse. However, the average distance is **384,400 kilometers (238,900 miles)**.\n\nHere's a bit more detail:\n\n*   **Perigee (closest point):** Around 363,104 kilometers (225,623 miles)\n*   **Apogee (farthest point):** Around 405,696 kilometers (252,088 miles)"}
